In [1]:
import os
import matplotlib.pyplot
import pandas as pd
import json
from glob import glob
from tqdm import tqdm
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
tqdm.pandas()

In [2]:
my_client_id = '4e9cf9cac4224e51b29649eb1ed2b916'
my_secret = '39313a32359e4963aa309b38b46c83fc'

aslan_client_id = '7e5000408e924c139e2f9ec522b3a012'
aslan_secret = '0ff1dc59346749068311a782ce80f7ca'

beka_client_id = 'fe22241e50584325acafcafc80a77adc'
beka_secret = 'e94196def67c46179f130e1800d134a6'

In [3]:
%env SPOTIPY_CLIENT_ID=fe22241e50584325acafcafc80a77adc
%env SPOTIPY_CLIENT_SECRET=e94196def67c46179f130e1800d134a6

env: SPOTIPY_CLIENT_ID=fe22241e50584325acafcafc80a77adc
env: SPOTIPY_CLIENT_SECRET=e94196def67c46179f130e1800d134a6


In [4]:
csv_path = "D:\projects\ProfanityFilter\spotify_songs.csv"
df = pd.read_csv(csv_path)
sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(),
    requests_timeout=20, 
    retries=10
)

# Find info about songs

In [5]:
def create_batch_df(df, batch_size, start):
    start_indices = range(start, len(df), batch_size);
    for s in start_indices:
        yield df[s:s+batch_size]
        
def find_tracks_explicit(tracks):
    info = sp.tracks(tracks)
    info_list = []
    for tr in info["tracks"]:
        if tr is not None:
            artists_uris = "|".join([a["uri"] for a in tr["artists"]])
            isrc = tr["external_ids"]["isrc"] if "isrc" in tr["external_ids"] else ""
            explicit = tr["explicit"] if "explicit" in tr else False
            info_instance = (
                tr["uri"],
                explicit,
                tr["popularity"],
                isrc,
                artists_uris
            )
            info_list.append(list(map(str, info_instance)))
        else:
            info_list.append(["","","","",""])
    return info_list

def save_to_file(savepath, ids, infos):
    with open(savepath, "a", encoding="utf-8") as wf:
        for i, infos in zip(ids, infos):
            els = [str(i)] + infos
            line = ','.join(els) + '\n'
            wf.write(line)
    return None

def find_explicit_songs(
    df, 
    bs=50, 
    start=0,
    savepath="D:\projects\ProfanityFilter\explicits\songs_info.txt"
):
    total = int((len(df) - start)/bs)
    pbar = tqdm(create_batch_df(df, bs, start), total=total)
    for i, b in enumerate(pbar):
        ids = b.index.tolist()
        uris = b["uri"].tolist()
        info_list = find_tracks_explicit(uris)
        save_to_file(savepath, ids, info_list)
    return ids[-1]

def find_last_idx(
    savepath="D:\projects\ProfanityFilter\explicits\songs_info.txt"
):
    if not os.path.isfile(savepath):
        return 0
    idx = open(savepath).readlines()[-1].split(",")[0]
    return int(idx)

In [6]:
find_last_idx()

1249259

In [7]:
last_idx = find_last_idx()
find_explicit_songs(df, start=last_idx, bs=50)

20261it [1:09:11,  4.88it/s]                                                                                                                           


2262291